## Basic Animal Classification using 

In [ ]:
import torch
import torchvision
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torchinfo import summary
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

### Animal-10 Dataset

The Animals-10 Dataset contains approximately 28,000 medium-quality images categorized into 10 classes: dog, cat, horse, spider, butterfly, chicken, sheep, cow, squirrel, and elephant. These images were collected from Google Images and verified by humans, with some erroneous samples included to reflect real-world conditions, such as user-submitted images.

The dataset is organized into directories corresponding to each category, with class sizes ranging from 2,000 to 5,000 images. This dataset can simulate applications such as smart image galleries for researchers like biologists.

This dataset was downloaded from Kaggle and was submitted by Corrado Alessio.

### Preparing and Splitting Dataset 

First, we define a manual transformation pipeline, which resizes images to 224x224 pixels and converts them to tensors. The dataset is loaded using ImageFolder with the defined transformation applied. The dataset is then split into three subsets: training, testing, and validation. An 80-20 split is first applied to separate the training set from a combined test and validation set. This combined set is further divided into two equal parts for testing and validation. This setup ensures the dataset is properly preprocessed and divided for model evaluation.

In [ ]:
manual_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [ ]:
base_dir = os.path.dirname('MCO')  
dataset_path = os.path.join(base_dir, 'archive', 'raw-img')

dataset = ImageFolder(root=dataset_path, transform=manual_transform)

In [ ]:
# Create train dataset 
train_size = int(0.8 * len(dataset))
test_val_size = len(dataset) - train_size
train_data, test_val_data = random_split(dataset, [train_size, test_val_size])

# Create test and validation set from test_val_dataset
test_size = int(0.5 * len(test_val_data))
val_size = len(test_val_data) - test_size
test_data, val_data = random_split(test_val_data, [test_size, val_size])

# Check the len of the train, test, val dataset
len(train_data), len(test_data), len(val_data)

### Setting Up Pre-trained EfficientNet-B0 with Data Transformations

This code sets up a pre-trained EfficientNet-B0 model using PyTorch's torchvision library. It loads default weights optimized on a large dataset (like ImageNet), prepares the model for the specified device (CPU or GPU), and retrieves the recommended data preprocessing steps (like resizing and normalization) that match the model's training. These transformations ensure the input data is properly formatted for accurate predictions.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

In [ ]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

In [ ]:
weights

In [ ]:
auto_transforms = weights.transforms()
auto_transforms

### Creating Dataset with Pretrained Transformations

Earlier, we did manual_transform which provided the basic preprocessing we defined relating to image size. Now, we will also apply auto_transform which was predefined and tailored to the selected pre-trained model (EfficientNet). 

In this case, as shown above, the images are resized so that their shorter edge is 256 pixels while maintaining the aspect ratio. They are then cropped to a fixed size of 224x224 pixels to match the input dimensions expected by the model.  Bicubic interpolation is used during resizing, which provides smooth and high-quality image scaling. 

The images are also normalized using mean values [0.485, 0.456, 0.406] and standard deviations [0.229, 0.224, 0.225] for the red, green, and blue channels, respectively. This normalization ensures the pixel values are scaled and centered around zero with a standard deviation of one, consistent with the dataset (e.g., ImageNet) on which the model was originally trained. These steps collectively ensure compatibility with the model and optimize its performance.

The dataset is then split into three subsets: training, testing, and validation. An 80-20 split is first applied to separate the training set from a combined test and validation set. This combined set is further divided into two equal parts for testing and validation. This setup ensures the dataset is properly preprocessed and divided for model evaluation.

In [ ]:
base_dir = os.path.dirname('MCO')  
dataset_path = os.path.join(base_dir, 'archive', 'raw-img')

dataset = ImageFolder(root=dataset_path, transform=auto_transforms)

In [ ]:
# Create train dataset
train_size = int(0.8 * len(dataset))
test_val_size = len(dataset) - train_size
train_dataset, test_val_dataset = random_split(dataset, [train_size, test_val_size])

# Create test and validation set from test_val_dataset
test_size = int(0.5 * len(test_val_dataset))
val_size = len(test_val_dataset) - test_size
test_dataset, val_dataset = random_split(test_val_dataset, [test_size, val_size])

# Check the len of the train, test, val dataset
len(train_dataset), len(test_dataset), len(val_dataset)

### Creating DataLoaders
 In this stage, data loaders for training, testing, and validation datasets are created. This enables efficient data batching, shuffling, and parallel data loading during training or evaluation. 
 
 The lengths of the data loaders indicate the number of batches in each dataset, which are (655, 82, 82) for train_loader, test_loader, and val_loader respectively.

In [ ]:
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=2)

test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=2)

val_loader = DataLoader(val_dataset,
                       batch_size=BATCH_SIZE,
                       shuffle=False,
                       num_workers=2)

len(train_loader), len(test_loader), len(val_loader)

### Visualizing the Dataset



In [ ]:
class_names = dataset.classes
class_names

In [ ]:
translate = {
    "cane": "dog", 
    "cavallo": "horse", 
    "elefante": "elephant", 
    "farfalla": "butterfly", 
    "gallina": "chicken", 
    "gatto": "cat", 
    "mucca": "cow", 
    "pecora": "sheep", 
    "scoiattolo": "squirrel", 
    "ragno": "spider",
    "dog": "cane", 
    "elephant": "elefante", 
    "butterfly": "farfalla", 
    "chicken": "gallina", 
    "cat": "gatto", 
    "cow": "mucca", 
    "spider": "ragno", 
    "squirrel": "scoiattolo"
}

english_class_names = [translate[class_name] for class_name in class_names]
english_class_names

In [ ]:
# Plot images
torch.manual_seed(42)
fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(train_dataset), size=[1]).item()
    img, label = train_dataset[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.permute(2, 1, 0))
    plt.title(english_class_names[label])
    plt.axis(False); 

The code below freezes the base feature extractor. This prevents updates to these layers during training, ensuring only the newly added classifier layers are trained.

In [ ]:
for param in model.features.parameters():
    param.requires_grad = False

The random number generators for both the CPU (torch.manual_seed) and GPU (torch.cuda.manual_seed) are seeded with the value 42 to ensure reproducibility of results by controlling the randomness in weight initialization and other stochastic processes.

The number of output units for the new classifier layer is determined based on the length of class_names, which corresponds to the number of target classes in the dataset.

The classifier section of the model is replaced with a new sequential layer. 

It includes:
- A Dropout layer with a dropout probability of 0.2 to help prevent overfitting.
- A Linear layer with 1280 input features and output_shape output units, where output_shape matches the number of classes. This layer is moved to the specified device (to(device)), such as a CPU or GPU.

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1280, 
                    out_features=output_shape,
                    bias=True)).to(device)

In [ ]:
summary(model=model, 
        input_size=(32, 3, 224, 224), 
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 

### Defining Loss and Optimizer

The selected loss function is Cross-Entropy Loss since it is suitable for multi-class classification problems where each input belongs to exactly one of the classes. It measures the difference between the predicted class probabilities (output from the model) and the true class labels.

The selected optimizer is Adam optimizer, a widely used optimization algorithm combining the benefits of both momentum-based optimization and adaptive learning rates. The learning rate is set to 0.001 which controls how much the model's weights are adjusted in response to the computed gradient.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

### Training and Testing the Model

Here, we set the random seed for reproducibility. A PyTorch model for 5 epochs using a training and validation dataset, Adam optimizer, and Cross Entropy Loss is trained. We also measure and print the total training time.